In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip 
# !{sys.executable} -m pip install pandas 
# !{sys.executable} -m pip install geopandas 
# !{sys.executable} -m pip install ipyleaflet 
# !{sys.executable} -m pip install bs4 
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install seaborn --user

In [2]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
from datetime import date
from app import RetrieveAndSave, DataPrep, PlotData, createFolder
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, GeoData, LayersControl

import ipywidgets as widgets 
from ipywidgets import AppLayout, Button, Layout, Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, DatePicker, Output, VBox, HBox, Text, SelectMultiple, Combobox, Accordion
from ipywidgets.embed import embed_data
from IPython.display import clear_output, HTML, display, Image

import threading
import time
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
import datetime
import itertools


In [3]:
raws_gdf = gpd.read_file('../data/weather_stations.geojson')

In [4]:
terrain_base = basemap_to_tiles(basemaps.Stamen.Terrain)

m = Map(layers=(terrain_base, ), center=(37.740294939381876, -120.57208675168276), zoom=8)

draw_control = DrawControl(rectangle={ "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity":0.4}})

feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

def handle_draw(self, action, geo_json):
    feature_collection['features'].append(geo_json)

draw_control.on_draw(handle_draw)

station_data = GeoData(geo_dataframe = raws_gdf,
                hover_style={'color' : 'red', 'fillColor': 'red' , 'fillOpacity': 0.8, 'weight': 1},
                point_style={'radius': 3, 'color': 'blue', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 1},
                name = 'station')

m.add_layer(station_data)
m.add_control(draw_control)

In [5]:
header = widgets.HTML("<h1>Weather Station Data Downloader</h1>", layout=Layout(height='auto'))
header.style.text_align='center'

footer = widgets.HTML('<h5 style="color:DodgerBlue;"> Source: RAWS US Climate Archive [raws.dri.edu/] </h5>', layout=Layout(height='auto'))

start_dt = DatePicker(layout=Layout(flex='1 1 0%', width='auto'), disabled=False)
end_dt = DatePicker(layout=Layout(flex='1 1 0%', width='auto'), disabled=False)

attributes_selector = SelectMultiple(
            rows=10,
            description='Attributes',
            disabled=False,
            options = ['Total_Solar_Rad', 'Wind_Avg_mph', 'Wind_Dir_Deg', 
                        'Wind_Max_mph', 'Air_Temp_Avg', 'Fuel_Temp_Avg', 'Fuel_Moist_Per',
                        'Rel_Hum_Per', 'Dew_Point_Deg', 'Wet_Bulb', 'Total_Precip']
        )

input_filename = widgets.Text(
    value='sample.csv',
    placeholder='filename',
    description='Save File as:',
    disabled=False, style=dict(description_width='initial')
    )

aggr_toggle = widgets.RadioButtons(
    options=['Hourly', 'Daily', 'Monthly', 'Annually'],
    description='Aggregation', value='Hourly',
    disabled=False, style=dict(description_width='initial')
    )

daytime_slider = widgets.SelectionRangeSlider(
    options=[i for i in range(0, 24)],
    index=(0, 23),
    description='Daytime Hours', style=dict(description_width='initial'),
    disabled=False
)

daytime_toggle = widgets.ToggleButtons(
    options=['None', 'Daytime Only', 'Nighttime Only', 'Day & Night'],
    value='Day & Night',
    description='Specification',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or '', icons=['check', 'check', 'check'], 
    style=dict(description_width='initial')
)

Request_button = widgets.Button(
    description='Download Data',
    disabled=False,
    button_style='success',
    icon='table', layout = Layout(width='200px', height='auto'),
    style=dict(description_width='initial'))


progress_search = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, 
    description='Progress',
    bar_style='info',
    style={'bar_color': '#94b79f'})
Complete_msg = widgets.HTML()

def work(progress_search):
    Complete_msg.value = ""
    total = 100
    for i in range(total):
        time.sleep(0.3)
        progress_search.value = float(i+1)/total
    Complete_msg.value = f"<h4 style='color:MediumSeaGreen;'> Complete! </h4>"

def callback(wdgt):
    thread = threading.Thread(target=work, args=(progress_search,))
    display(progress_search)
    thread.start()
    data_raw = RetrieveAndSave(start_dt.value, end_dt.value, raws_gdf, csv_name = input_filename.value, filter_by = attributes_selector.value, aggr_by = aggr_toggle.value).retrieveRAWS_saveIntoCSV(feature_collection)
    

tab1_header = widgets.HTML(value='<h2>Weather Station Downloader</h2>')


tab1 = HBox(children=[VBox(children=[tab1_header, HBox(children=[Label(value='Start Date', layout=dict(height='auto')), start_dt, Label(value='End Date', layout=dict(height='auto')), end_dt]), attributes_selector, input_filename, Request_button, HBox(children=[progress_search, Complete_msg]), footer]), m ])

m.layout.width='auto'
m.layout.height='auto'

Request_button.on_click(callback)

In [6]:
from os import listdir
from os.path import isfile, join


def get_files(mypath = os.path.join('../data', 'RAWS_CSV')):
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    return onlyfiles

def get_vars(input_filenm): # selection_filename.value 
    data = pd.read_csv(os.path.join('../data', 'RAWS_CSV', input_filenm))
    avail_vars = list(data.columns.values)[3:]
    return avail_vars

def get_stations(input_filenm): # selection_filename.value 
    data = pd.read_csv(os.path.join('../data', 'RAWS_CSV', input_filenm))
    avail_stations = list(data['Station Name'].unique()) 
    return avail_stations

In [13]:
map_output = widgets.Output(layout = Layout(width='600px', height='350px'))
plot_output = widgets.Output(layout = Layout(width='600px', height='auto'))

selection_filename = widgets.Dropdown(
    options=get_files(),
    description='Select File:',
    style=dict(description_width='initial')
)

select_var = widgets.Dropdown(
    options=get_vars(input_filenm = selection_filename.value),
    description='Select Variable:',
    style=dict(description_width='initial')
)

select_station = widgets.Dropdown(
    options=get_stations(input_filenm = selection_filename.value),
    description='Select Station:',
    style=dict(description_width='initial')
)

def on_value_change(change):
    select_var.options = get_vars(input_filenm = selection_filename.value)
    select_station.options = get_stations(input_filenm = selection_filename.value)

def plot(df):
    sns.set_style('whitegrid', {'legend.frameon':True, 'framealpha':1})
    plt.figure(figsize=(10,5))
    filter_df = df[df['Station Name'] == select_station.value]
    chart = sns.lineplot(data= filter_df, x="Datetime", y=select_var.value)
    chart.set(xticks=df.Datetime[2::8])
    plt.title(f'Timeseries: {select_var.value} for {select_station.value} Station')
    plt.xticks(
        rotation=80, 
        horizontalalignment='right',
        fontweight='light',
        fontsize='x-small' )
    None
    createFolder('../data', 'timeseries_plots')
    plt.savefig(os.path.join('../data', 'timeseries_plots', 'chart.png'), dpi=400)

def on_station_change(change):
    with map_output: 
        clear_output()
        WS_location = Map(layers=(terrain_base, ), center=get_station_coord(select_station.value), zoom=11)
        WS_location.add_layer(Marker(location=get_station_coord(select_station.value)))
        display(WS_location)


def plotAction(wdgt):
    #df = DataPrep(selection_filename.value, daytime_slider.value, daytime_toggle.value, aggr_toggle.value).filter_byVar()
    
    df = pd.read_csv(os.path.join('../data', 'RAWS_CSV', selection_filename.value))
    with plot_output: 
        clear_output()
        plot(df)
        img = Image(os.path.join('../data', 'timeseries_plots', 'chart.png'))
        display(img)
    
        

Plot_button = widgets.Button(
    description='Plot Data',
    disabled=False,
    button_style='success',
    icon='signal', layout = Layout(width='200px', height='auto'),
    style=dict(description_width='initial'))


Save_Plot = widgets.Button(
    description='Download',
    disabled=False,
    button_style='Danger',
    icon='share', layout = Layout(width='200px', height='auto'),
    style=dict(description_width='initial'))


selection_filename.observe(on_value_change, names='value')
select_station.observe(on_station_change, names='value')
Plot_button.on_click(plotAction)


In [14]:
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles
terrain_base = basemap_to_tiles(basemaps.Stamen.Terrain)

def get_station_coord(station_name):
    raws_info = pd.read_csv("../data/RAWS_info_cleaned.csv")
    selected_info = raws_info[raws_info['station'].str.strip() == station_name]
    lat = selected_info['decimal_lat'].iloc[0]
    lon = selected_info['decimal_long'].iloc[0]
    coord = (lat, lon)
    return coord 

    

In [21]:
tab2_header = widgets.HTML(value='<h2>Data Visualizer</h2>')

tab2 = HBox(children=[VBox(children=[tab2_header, selection_filename, daytime_toggle, daytime_slider, aggr_toggle, select_var, select_station, Plot_button]), map_output])

tab3_header = widgets.HTML(value='<h2>Output Chart</h2>')
tab3 = VBox(children=[plot_output, Save_Plot])
#Save_Plot

In [22]:
tab_nest = widgets.Tab()
tab_nest.children = [tab1, tab2, tab3]
tab_nest.set_title(0, 'Download RAWS')
tab_nest.set_title(1, 'Visualize')
tab_nest.set_title(2, 'Results')
tab_nest

In [11]:
#DataPrep(selection_filename.value, daytime_slider.value, daytime_toggle.value, aggr_toggle.value).filter_byVar()
df = pd.read_csv(os.path.join('../data', 'RAWS_CSV', 'sample1.csv'))
PlotData(df, select_station.value, select_var.value)

In [12]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid', {'legend.frameon':True, 'framealpha':1})
plt.figure(figsize=(10,5))
filter_df = df[df['Station Name'] == select_station.value]
chart = sns.lineplot(data= filter_df, x="Datetime", y=select_var.value)

chart.set(xticks=df.Datetime[2::8])
plt.xticks(
    rotation=80, 
    horizontalalignment='right',
    fontweight='light',
    fontsize='x-small' )
None

ValueError: Could not interpret value `Total_Solar_Rad` for parameter `y`